<a href="https://colab.research.google.com/github/AntonG7/hack1/blob/master/scrap_wikitable.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scrap une table de Wikipedia. 
Prend en compte des cellules qui s'étendent sur plusieurs lignes/colonnes.
La sortie est un fichiers csv avec les éléments sélectionnés



Imports


In [1]:
# -*- coding: utf-8 -*-
from bs4 import BeautifulSoup
import requests
import os
import codecs

Parametres Wikipedia 

In [3]:
wiki = "https://fr.wikipedia.org/wiki/Liste_des_d%C3%A9partements_fran%C3%A7ais_class%C3%A9s_par_population_et_superficie"
header = {
    'User-Agent': 'Mozilla/5.0'
}  # Needed to prevent 403 error on Wikipedia
page = requests.get(wiki, headers=header)
soup = BeautifulSoup(page.content)
tables = soup.findAll("table", {"class": "wikitable"})

In [ ]:
# show tables
for i, table in enumerate(tables):
    print("#"*10 + "Table {}".format(i) + '#'*10)
    print(table.text[:100])
    print('.'*80)
print("#"*80)

Détection des cellules avec balises HTML

In [ ]:
for tn, table in enumerate(tables):

    # preinit list of lists
    rows = table.findAll("tr")
    row_lengths = [len(r.findAll(['th', 'td'])) for r in rows]
    ncols = max(row_lengths)
    nrows = len(rows)
    data = []
    for i in range(nrows):
        rowD = []
        for j in range(ncols):
            rowD.append('')
        data.append(rowD)

    # process html
    for i in range(len(rows)):
        row = rows[i]
        rowD = []
        cells = row.findAll(["td", "th"])
        for j in range(len(cells)):
            cell = cells[j]

            #lots of cells span cols and rows so lets deal with that
            cspan = int(cell.get('colspan', 1))
            rspan = int(cell.get('rowspan', 1))
            l = 0
            for k in range(rspan):
                # Shifts to the first empty cell of this row
                while data[i + k][j + l]:
                    l += 1
                for m in range(cspan):
                    cell_n = j + l + m
                    row_n = i + k
                    # in some cases the colspan can overflow the table, in those cases just get the last item
                    cell_n = min(cell_n, len(data[row_n])-1)
                    data[row_n][cell_n] += cell.text
                    print(cell.text)

data.append(rowD)

In [6]:
len(data)

106

Sélection des éléments

In [32]:
code_insee = []
departement = []
population = []
for i in range(2, len(data)-1):
  code_insee.append(data[i][1])
  departement.append(data[i][2])
  population.append(data[i][10])
for i in range(0, len(departement)):
  print('insee: ',code_insee[i], 'dep: ',departement[i], ', pop: ', population[i])


insee:  59 dep:  Nord , pop:  2 604 361
insee:  75 dep:  Paris , pop:  2 187 526
insee:  13 dep:  Bouches-du-Rhône , pop:  2 024 162
insee:  69 dep:  Circonscription départementale du Rhône , pop:  1 843 319
insee:  93 dep:  Seine-Saint-Denis , pop:  1 623 111
insee:  92 dep:  Hauts-de-Seine , pop:  1 609 306
insee:  33 dep:  Gironde , pop:  1 583 384
insee:  62 dep:  Pas-de-Calais , pop:  1 468 018
insee:  78 dep:  Yvelines , pop:  1 438 266
insee:  77 dep:  Seine-et-Marne , pop:  1 403 997
insee:  44 dep:  Loire-Atlantique , pop:  1 394 909
insee:  94 dep:  Val-de-Marne , pop:  1 387 926
insee:  69M dep:  Métropole de Lyon , pop:  
insee:  31 dep:  Haute-Garonne , pop:  1 362 672
insee:  91 dep:  Essonne , pop:  1 296 130
insee:  38 dep:  Isère , pop:  1 258 722
insee:  76 dep:  Seine-Maritime , pop:  1 254 378
insee:  95 dep:  Val-d'Oise , pop:  1 228 618
insee:  34 dep:  Hérault , pop:  1 144 892
insee:  67 dep:  Bas-Rhin , pop:  1 125 559
insee:  06 dep:  Alpes-Maritimes , pop:  1

Dataframe final

In [33]:
import pandas as pd
df = pd.DataFrame()

df['code_insee'] = code_insee
df['departement'] = departement
df['population'] = population

In [34]:
df

,code_insee,departement,population
0,59,Nord,2 604 361
1,75,Paris,2 187 526
2,13,Bouches-du-Rhône,2 024 162
3,69,Circonscription départementale du Rhône,1 843 319
4,93,Seine-Saint-Denis,1 623 111
...,...,...,...
98,15,Cantal,145 143
99,90,Territoire de Belfort,142 622
100,05,Hautes-Alpes,141 284
101,23,Creuse,118 638


Ecriture du fichier de sortie

In [37]:
df.to_csv('population_2017_departement.csv', encoding='utf-8')
